In [ ]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import sys
import warnings
import matplotlib.pyplot as plt
import random
warnings.filterwarnings('ignore')

In [ ]:
import torch.nn.functional as F
import torch.nn as nn
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.bn2d1 = nn.BatchNorm2d(6, affine=True)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.bn2d2 = nn.BatchNorm2d(16, affine=True)
        self.fc1   = nn.Linear(16*5*5, 120)
        self.bn1   = nn.BatchNorm1d(120, affine=True)
        self.fc2   = nn.Linear(120, 84)
        self.bn2   = nn.BatchNorm1d(84, affine=True)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        conv1 = F.relu(self.bn2d1(self.conv1(x)))
        conv1 = F.max_pool2d(conv1, 2)
        conv2 = F.relu(self.bn2d2(self.conv2(conv1)))
        conv2 = F.max_pool2d(conv2, 2)
        fc = conv2.view(conv2.size(0), -1)
        h_relu1 = self.bn1(self.fc1(fc)).clamp(min=0)
        h_relu2 = self.bn2(self.fc2(h_relu1)).clamp(min=0)
        out = self.fc3(h_relu2)
        return (out, h_relu1, h_relu2)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)[0]
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
k_mnist_train = datasets.KMNIST("../data", train=True, download=True, transform=transforms.ToTensor())
k_mnist_test = datasets.KMNIST("../data", train=False, download=True, transform=transforms.ToTensor())
k_train_loader = DataLoader(k_mnist_train, batch_size = 64, shuffle=True)
k_test_loader = DataLoader(k_mnist_test, batch_size = 128, shuffle=False)

In [ ]:
k_mnist_test.data.shape

In [ ]:
image_datasets = {'train':k_mnist_train,'val':k_mnist_test}
dataloaders = {'train':k_train_loader,'val':k_test_loader}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
model = LeNet5()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

In [ ]:
torch.save(model_ft.state_dict(), './trained_model/LeNet_kmnist.pt')

In [ ]:
def accu(model, dataloader):
    model = model.eval()
    acc = 0
    total = 0
    for data, target in dataloader:
        o = model(data.to(device))[0]
        total += target.to(device).size(0)
        acc += (o.argmax(dim=1).long() == target.to(device)).sum().item()
    return acc / total

In [ ]:
f_mnist_train = datasets.FashionMNIST("../data", train=True, download=True, transform=transforms.ToTensor())
f_mnist_test = datasets.FashionMNIST("../data", train=False, download=True, transform=transforms.ToTensor())
f_train_loader = DataLoader(f_mnist_train, batch_size = 64, shuffle=True)
f_test_loader = DataLoader(f_mnist_test, batch_size = 128, shuffle=False)

In [ ]:
accu(model_ft, f_train_loader)

In [ ]:
accu(model_ft, f_test_loader)

In [ ]:
features = []
tags = []
with torch.no_grad():
    for data in f_train_loader:
        images, labels = data
        outputs = model_ft(images.to(device))[2]
        features.append(outputs.cpu().data.numpy())
        tags.append(labels.cpu().data.numpy())

train_feature = pd.concat([pd.DataFrame(x) for x in features])
train_label = np.concatenate(tags)
train_label = pd.Series(train_label)

train_feature.to_csv('./processed_data/fmnist_kmnist_train.csv',index=False,header=False)
train_label.to_csv('./processed_data/fmnist_kmnist_train_label.csv',index=False,header=False)

In [ ]:
features = []
tags = []
with torch.no_grad():
    for data in f_test_loader:
        images, labels = data
        outputs = model_ft(images.to(device))[2]
        features.append(outputs.cpu().data.numpy())
        tags.append(labels.cpu().data.numpy())

train_feature = pd.concat([pd.DataFrame(x) for x in features])
train_label = np.concatenate(tags)
train_label = pd.Series(train_label)

train_feature.to_csv('./processed_data/fmnist_kmnist_test.csv',index=False,header=False)
train_label.to_csv('./processed_data/fmnist_kmnist_test_label.csv',index=False,header=False)

In [ ]:
mnist_train = datasets.MNIST("../data", train=True, download=True, transform=transforms.ToTensor())
mnist_test = datasets.MNIST("../data", train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(mnist_train, batch_size = 64, shuffle=False)
test_loader = DataLoader(mnist_test, batch_size = 128, shuffle=False)

In [ ]:
accu(model_ft, train_loader)

In [ ]:
accu(model_ft, test_loader)

In [ ]:
features = []
tags = []
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        outputs = model_ft(images.to(device))[2]
        features.append(outputs.cpu().data.numpy())
        tags.append(labels.cpu().data.numpy())

train_feature = pd.concat([pd.DataFrame(x) for x in features])
train_label = np.concatenate(tags)
train_label = pd.Series(train_label)

train_feature.to_csv('./processed_data/mnist_kmnist_train.csv',index=False,header=False)
train_label.to_csv('./processed_data/mnist_kmnist_train_label.csv',index=False,header=False)

In [ ]:
features = []
tags = []
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model_ft(images.to(device))[2]
        features.append(outputs.cpu().data.numpy())
        tags.append(labels.cpu().data.numpy())

train_feature = pd.concat([pd.DataFrame(x) for x in features])
train_label = np.concatenate(tags)
train_label = pd.Series(train_label)

train_feature.to_csv('./processed_data/mnist_kmnist_test.csv',index=False,header=False)
train_label.to_csv('./processed_data/mnist_kmnist_test_label.csv',index=False,header=False)